In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
file_names = [f'/content/drive/MyDrive/RegressionProject/listings ({i}).csv' for i in range(1, 6)]
dataframes = [pd.read_csv(file_name) for file_name in file_names]
listings = pd.concat(dataframes, ignore_index=True)

In [ ]:
len(listings['id'].unique())

49759

In [ ]:
listings['price'] = listings['price'].str.replace(r'[^0-9.]', '', regex=True).astype(float)
listings['reviews_per_month'] = listings['reviews_per_month'].fillna(0)  # Assuming no reviews means 0

In [ ]:
listings = listings[['id', 'price', 'availability_365', 'reviews_per_month']]

In [ ]:
listings = listings.sample(frac=0.4)

In [ ]:
listings['reviews_per_month'].value_counts()

0.00     21099
0.02      2179
0.01      2091
0.04      1645
0.03      1582
         ...  
11.01        1
11.50        1
41.69        1
8.40         1
12.53        1
Name: reviews_per_month, Length: 1003, dtype: int64

In [ ]:
listings = listings[listings['reviews_per_month'] != 0]

In [ ]:
listings.dtypes

id                     int64
price                float64
availability_365       int64
reviews_per_month    float64
dtype: object

In [ ]:
average_nights = 4.5
review_rate = 0.5
max_occupancy_rate = 0.95

listings['booking_per_month'] = listings['reviews_per_month'] / review_rate

listings['estimated_occupancy'] = (listings['booking_per_month'] * average_nights / 30).clip(upper=max_occupancy_rate)

listings['estimated_nights_per_year'] = listings['estimated_occupancy'] * listings['availability_365']

result_columns = ['id', 'booking_per_month', 'estimated_occupancy', 'estimated_nights_per_year', 'price']
results_df = listings[result_columns]

print(results_df.head())

              id  booking_per_month  estimated_occupancy  \
150004   6421011               0.10                0.015   
105359  40152377               0.44                0.066   
154029  13174320               0.10                0.015   
123976   3349699               0.82                0.123   
139569  17211583               3.04                0.456   

        estimated_nights_per_year  price  
150004                      0.000  110.0  
105359                      0.000   50.0  
154029                      0.000   60.0  
123976                     22.878  150.0  
139569                    154.128   65.0  


In [ ]:
import os
folder_path = '/content/drive/MyDrive/RegressionProject/Clustering/CSV Files/'
files = os.listdir(folder_path)

In [ ]:
dfs = []
for file in files:
    if file.endswith('.csv'):  # Check for CSV files
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path)
        dfs.append(df)

# Merge all dataframes into one
merged_df = pd.concat(dfs, ignore_index=True)

In [ ]:
merged_df = merged_df.drop_duplicates(subset='id', keep='first')

In [ ]:
merged_df

,id,latitude,longitude,CLUSTER
0,12191010,40.58893,-73.80064,1
1,12205604,40.58866,-73.79367,0
2,12221970,40.59349,-73.78829,1
3,12272376,40.59367,-73.78870,1
4,13524592,40.59209,-73.79045,1
...,...,...,...,...
855651,584641571670399489,40.74275,-73.89314,1
855652,582789867315606351,40.74268,-73.89198,1
855653,591126001314853341,40.75428,-73.90173,0
855654,596511810816093452,40.74209,-73.90220,2


In [ ]:
results_df

,id,booking_per_month,estimated_occupancy,estimated_nights_per_year,price
150004,6421011,0.10,0.015,0.000,110.0
105359,40152377,0.44,0.066,0.000,50.0
154029,13174320,0.10,0.015,0.000,60.0
123976,3349699,0.82,0.123,22.878,150.0
139569,17211583,3.04,0.456,154.128,65.0
...,...,...,...,...,...
43741,4467771,3.24,0.486,0.486,365.0
19897,46414174,2.82,0.423,0.000,218.0
108229,28184199,0.10,0.015,0.000,100.0
96909,621077359785532254,0.32,0.048,12.912,350.0


In [ ]:
combined_df = pd.merge(merged_df, results_df, on='id')

In [ ]:
combined_df = combined_df.drop_duplicates(subset='id', keep='first')

In [ ]:
combined_df.isnull().sum()

id                           0
latitude                     0
longitude                    0
CLUSTER                      0
booking_per_month            0
estimated_occupancy          0
estimated_nights_per_year    0
price                        0
dtype: int64

In [ ]:
combined_df = combined_df[combined_df['price'] <= 500]

In [ ]:
combined_df

,id,latitude,longitude,CLUSTER,booking_per_month,estimated_occupancy,estimated_nights_per_year,price
3,12272376,40.59367,-73.78870,1,3.82,0.573,204.561,150.0
5,13524592,40.59209,-73.79045,1,0.52,0.078,28.470,41.0
8,13525009,40.59252,-73.78853,1,0.82,0.123,44.895,36.0
9,13556488,40.58813,-73.79653,1,0.04,0.006,0.000,115.0
10,13657292,40.59444,-73.80159,1,1.14,0.171,14.364,230.0
...,...,...,...,...,...,...,...,...
48901,586244870263367585,40.74448,-73.89627,2,4.24,0.636,224.508,82.0
48903,584641571670399489,40.74275,-73.89314,1,2.86,0.429,21.021,147.0
48905,582789867315606351,40.74268,-73.89198,1,6.42,0.950,155.800,80.0
48906,596511810816093452,40.74209,-73.90220,2,15.84,0.950,151.050,70.0


In [ ]:
df = combined_df.copy()

In [ ]:
df

,id,latitude,longitude,CLUSTER,booking_per_month,estimated_occupancy,estimated_nights_per_year,price
3,12272376,40.59367,-73.78870,1,3.82,0.573,204.561,150.0
5,13524592,40.59209,-73.79045,1,0.52,0.078,28.470,41.0
8,13525009,40.59252,-73.78853,1,0.82,0.123,44.895,36.0
9,13556488,40.58813,-73.79653,1,0.04,0.006,0.000,115.0
10,13657292,40.59444,-73.80159,1,1.14,0.171,14.364,230.0
...,...,...,...,...,...,...,...,...
48901,586244870263367585,40.74448,-73.89627,2,4.24,0.636,224.508,82.0
48903,584641571670399489,40.74275,-73.89314,1,2.86,0.429,21.021,147.0
48905,582789867315606351,40.74268,-73.89198,1,6.42,0.950,155.800,80.0
48906,596511810816093452,40.74209,-73.90220,2,15.84,0.950,151.050,70.0


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression


# Dictionary to store models for each cluster
models = {}

# Fit a model for each cluster and calculate D_xij
for cluster in df['CLUSTER'].unique():
    # Filter data for the current cluster
    cluster_data = df[df['CLUSTER'] == cluster]

    # Extract independent and dependent variables
    X = cluster_data[['price']]
    y = cluster_data['estimated_occupancy']

    # Polynomial features
    poly = PolynomialFeatures(degree=2, include_bias=False)
    X_poly = poly.fit_transform(X)

    # Fit the model
    model = LinearRegression()
    model.fit(X_poly, y)

    # Store the model
    models[cluster] = model

    # Calculate D_xij for the cluster
    df.loc[df['CLUSTER'] == cluster, 'D_xij'] = model.predict(X_poly)


# Display the model coefficients for each cluster (optional)
for cluster, model in models.items():
    print(f"Cluster {cluster}:")
    print(f"Intercept: {model.intercept_}")
    print(f"Coefficients: {model.coef_}")
    print()

# Show the DataFrame with D_xij values
df.head()


Cluster 1:
Intercept: 0.15475828793149932
Coefficients: [ 8.59965917e-04 -1.28366128e-06]

Cluster 0:
Intercept: 0.13022947478410168
Coefficients: [ 1.68965886e-03 -3.01437565e-06]

Cluster 2:
Intercept: 0.1424583695397679
Coefficients: [ 1.50336630e-03 -2.64082121e-06]



,id,latitude,longitude,CLUSTER,booking_per_month,estimated_occupancy,estimated_nights_per_year,price,D_xij
3,12272376,40.59367,-73.78870,1,3.82,0.573,204.561,150.0,0.254871
5,13524592,40.59209,-73.79045,1,0.52,0.078,28.470,41.0,0.187859
8,13525009,40.59252,-73.78853,1,0.82,0.123,44.895,36.0,0.184053
9,13556488,40.58813,-73.79653,1,0.04,0.006,0.000,115.0,0.236678
10,13657292,40.59444,-73.80159,1,1.14,0.171,14.364,230.0,0.284645


In [ ]:
df_new = pd.read_csv('/content/drive/MyDrive/RegressionProject/predictions_df.csv')

In [ ]:
df_new = df_new[['property_category_svc_apt_hotel',
       'property_category_hostel', 'bedrooms', 'accommodates',
       'host_response_time_within_an_hour', 'gym', 'garden', 'pool', 'beds',
       'property_category_apartment', 'cooking_basics', 'minimum_nights',
       'washer', 'distance_to_nyc', 'id', 'predictions']]

In [ ]:
df = pd.merge(df, df_new, on='id')

In [ ]:
df['predictions'] = np.exp(df['predictions'])

In [ ]:
df.columns

Index(['id', 'latitude', 'longitude', 'CLUSTER', 'booking_per_month',
       'estimated_occupancy', 'estimated_nights_per_year', 'price', 'D_xij',
       'property_category_svc_apt_hotel', 'property_category_hostel',
       'bedrooms', 'accommodates', 'host_response_time_within_an_hour', 'gym',
       'garden', 'pool', 'beds', 'property_category_apartment',
       'cooking_basics', 'minimum_nights', 'washer', 'distance_to_nyc',
       'predictions'],
      dtype='object')

In [ ]:
from pyomo.environ import *
from pyomo.opt import SolverFactory
import pandas as pd

# Define the model
model = ConcreteModel()

# Define sets (assuming clusters are already defined in your DataFrame)
model.clusters = Set(initialize=df['CLUSTER'].unique())
model.days = RangeSet(1, 365)  # Assuming 365 days in a year

In [ ]:
# Define decision variables
model.x = Var(model.clusters, model.days, within=PositiveReals)  # Prices
model.alpha = Var(model.clusters, model.days, within=Binary)  # Booking decisions

In [ ]:
# Define parameters from the DataFrame
def baseline_price_rule(model, cluster, day):
    # Extract baseline price for each cluster. Modify according to your data structure.
    return df[df['CLUSTER'] == cluster]['predictions'].iloc[0]

def demand_rule(model, cluster, day):
    # Extract demand for each cluster. Modify according to your data structure.
    return df[df['CLUSTER'] == cluster]['D_xij'].iloc[0]

In [ ]:
model.x0 = Param(model.clusters, model.days, initialize=baseline_price_rule)

In [ ]:
model.D = Param(model.clusters, model.days, initialize=demand_rule)


In [ ]:
Cv = 20.0  # Variable cost
I0 = 300.0  # Initial investment per cluster

In [ ]:
def objective_rule(m):
    return sum(m.alpha[cluster, day] * (m.x[cluster, day] - Cv) for cluster in m.clusters for day in m.days) - I0

In [ ]:
model.objective = Objective(rule=objective_rule, sense=maximize)

In [ ]:
# Define constraints
def price_constraint(m, cluster, day):
    return m.x[cluster, day] == m.alpha[cluster, day] * m.x0[cluster, day]


In [ ]:
model.price_con = Constraint(model.clusters, model.days, rule=price_constraint)

In [ ]:
def demand_constraint(m, cluster, day):
    return m.D[cluster, day] >= 0.5 * m.alpha[cluster, day]

In [ ]:
model.demand_con = Constraint(model.clusters, model.days, rule=demand_constraint)

In [ ]:
!apt-get install -y -qq coinor-cbc


In [ ]:
!cbc -h


Welcome to the CBC MILP Solver 
Version: 2.10.7 
Build Date: Feb 14 2022 

command line - cbc -h (default strategy 1)
Multiple matches for h - possible completions:
heur(isticsOnOff)
help
hOp(tions)
hot(StartMaxIts)
CoinSolver takes input from arguments ( - switches to stdin)
Enter ? for list of commands or help
Coin:^C


In [ ]:
!pip install pyomo[solvers]

In [ ]:
!apt-get install -y -qq coinor-libipopt-dev
!pip install cyipopt

Selecting previously unselected package libscalapack-openmpi2.1:amd64.
(Reading database ... 120948 files and directories currently installed.)
Preparing to unpack .../00-libscalapack-openmpi2.1_2.1.0-4_amd64.deb ...
Unpacking libscalapack-openmpi2.1:amd64 (2.1.0-4) ...
Selecting previously unselected package libmumps-5.4:amd64.
Preparing to unpack .../01-libmumps-5.4_5.4.1-2_amd64.deb ...
Unpacking libmumps-5.4:amd64 (5.4.1-2) ...
Selecting previously unselected package libmumps-headers-dev.
Preparing to unpack .../02-libmumps-headers-dev_5.4.1-2_all.deb ...
Unpacking libmumps-headers-dev (5.4.1-2) ...
Selecting previously unselected package libscotch-6.1:amd64.
Preparing to unpack .../03-libscotch-6.1_6.1.3-1_amd64.deb ...
Unpacking libscotch-6.1:amd64 (6.1.3-1) ...
Selecting previously unselected package libmumps-seq-5.4:amd64.
Preparing to unpack .../04-libmumps-seq-5.4_5.4.1-2_amd64.deb ...
Unpacking libmumps-seq-5.4:amd64 (5.4.1-2) ...
Selecting previously unselected package coin

In [ ]:
!pip install pyomo
from pyomo.environ import *
import matplotlib.pyplot as plt
!wget -N -q "https://matematica.unipv.it/gualandi/solvers/ipopt-linux64.zip"
!unzip -o -q ipopt-linux64

In [ ]:
# Use IPOPT solver
solver = SolverFactory('ipopt', executable='/content/ipopt')
results = solver.solve(model)


In [1]:
# # Display results
# for cluster in model.clusters:
#     for day in model.days:
#         print(f'Cluster {cluster}, Day {day}: Price = {model.x[cluster,day].value}, Booking Decision = {model.alpha[cluster,day].value}')